In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [0]:
spark = SparkSession.builder.appName('Pepsico interview qs').getOrCreate()

In [0]:
data = [(1,'2024-01-01','I1',10,1000),(2,'2024-01-15','I2',20,2000),(3,'2024-02-01','I3',10,1500),(4,'2024-02-15','I4',20,2500),
        (5,'2024-03-01','I5',30,3000),(6,'2024-03-10','I6',40,3500),(7,'2024-03-20','I7',20,2500),(8,'2024-03-30','I8',10,1000)]

In [0]:
schema = ["SOId","SODate","ItemId","ItemQty","ItemValue"]

In [0]:
df = spark.createDataFrame(data,schema)

In [0]:
df.show()

+----+----------+------+-------+---------+
|SOId|    SODate|ItemId|ItemQty|ItemValue|
+----+----------+------+-------+---------+
|   1|2024-01-01|    I1|     10|     1000|
|   2|2024-01-15|    I2|     20|     2000|
|   3|2024-02-01|    I3|     10|     1500|
|   4|2024-02-15|    I4|     20|     2500|
|   5|2024-03-01|    I5|     30|     3000|
|   6|2024-03-10|    I6|     40|     3500|
|   7|2024-03-20|    I7|     20|     2500|
|   8|2024-03-30|    I8|     10|     1000|
+----+----------+------+-------+---------+



In [0]:
# type cast the SODate as DateType from String
df1 = df.withColumn('SODate',df.SODate.cast(DateType()))
df1.printSchema()
df.show()

root
 |-- SOId: long (nullable = true)
 |-- SODate: date (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- ItemQty: long (nullable = true)
 |-- ItemValue: long (nullable = true)

+----+----------+------+-------+---------+
|SOId|    SODate|ItemId|ItemQty|ItemValue|
+----+----------+------+-------+---------+
|   1|2024-01-01|    I1|     10|     1000|
|   2|2024-01-15|    I2|     20|     2000|
|   3|2024-02-01|    I3|     10|     1500|
|   4|2024-02-15|    I4|     20|     2500|
|   5|2024-03-01|    I5|     30|     3000|
|   6|2024-03-10|    I6|     40|     3500|
|   7|2024-03-20|    I7|     20|     2500|
|   8|2024-03-30|    I8|     10|     1000|
+----+----------+------+-------+---------+



In [0]:
from pyspark.sql.functions import *


In [0]:
df2 = df1.select(month(df1.SODate).alias('Month'),year(df1.SODate).alias('Year'),df1.ItemValue)

In [0]:
df2.show()

+-----+----+---------+
|Month|Year|ItemValue|
+-----+----+---------+
|    1|2024|     1000|
|    1|2024|     2000|
|    2|2024|     1500|
|    2|2024|     2500|
|    3|2024|     3000|
|    3|2024|     3500|
|    3|2024|     2500|
|    3|2024|     1000|
+-----+----+---------+



In [0]:
df3 = df2.groupBy(df2.Month,df2.Year).agg(sum(df2.ItemValue).alias('Total_value'))

In [0]:
df3.show()

+-----+----+-----------+
|Month|Year|Total_value|
+-----+----+-----------+
|    1|2024|       3000|
|    2|2024|       4000|
|    3|2024|      10000|
+-----+----+-----------+



In [0]:
from pyspark.sql.window import *

In [0]:
df4 = df3.select(df3.Month,df3.Year,df3.Total_value,lag(df3.Total_value).over(Window.orderBy(df3.Month,df3.Year)).alias('PrevSales'))

In [0]:
df4.show()

+-----+----+-----------+---------+
|Month|Year|Total_value|PrevSales|
+-----+----+-----------+---------+
|    1|2024|       3000|     NULL|
|    2|2024|       4000|     3000|
|    3|2024|      10000|     4000|
+-----+----+-----------+---------+



In [0]:
df5 = df4.select('*',(df4.Total_value - df4.PrevSales)*100/df4.Total_value).alias('Percenatge')

In [0]:
df5.show()


+-----+----+-----------+---------+-------------------------------------------------+
|Month|Year|Total_value|PrevSales|(((Total_value - PrevSales) * 100) / Total_value)|
+-----+----+-----------+---------+-------------------------------------------------+
|    1|2024|       3000|     NULL|                                             NULL|
|    2|2024|       4000|     3000|                                             25.0|
|    3|2024|      10000|     4000|                                             60.0|
+-----+----+-----------+---------+-------------------------------------------------+



In [0]:
df5.show()

+-----+----+-----------+---------+-------------------------------------------------+-------------------------------------------------+-------------------------------------------------+-------------------------------------------------+
|Month|Year|Total_value|PrevSales|(((Total_value - PrevSales) * 100) / Total_value)|(((Total_value - PrevSales) * 100) / Total_value)|(((Total_value - PrevSales) * 100) / Total_value)|(((Total_value - PrevSales) * 100) / Total_value)|
+-----+----+-----------+---------+-------------------------------------------------+-------------------------------------------------+-------------------------------------------------+-------------------------------------------------+
|    1|2024|       3000|     NULL|                                             NULL|                                             NULL|                                             NULL|                                             NULL|
|    2|2024|       4000|     3000|                          